# **SENTIMENT - ANALYSIS / USERS_REVIEWS**

## 1. Importación de Librerías

In [1]:
%pip install vaderSentiment

   ---------------------------------------- 0.0/126.0 kB ? eta -:--:--
   ---------------------- ----------------- 71.7/126.0 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 126.0/126.0 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ---------------------------------------- 62.6/62.6 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/163.8 kB ? eta -:--:--
   ---------------------------------------- 163.8/163.8 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/100.4 kB ? eta -:--:--
   ---------------------------------------- 100.4/100.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/61.6 kB ? eta -:--:--
   ---------------------------------------- 61.6/61.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/121.1 kB ? eta -:--:--
   ---------------------------------------- 121.1/121.1 kB 7.4 MB/s eta 0:00:00
Note: you may need to restart the k

In [2]:
"""

La librería vaderSentiment es una herramienta para análisis de sentimientos (sentiment analysis) en texto.
El nombre "VADER" es un acrónimo que significa "Valence Aware Dictionary and sEntiment Reasoner".
Fue diseñada específicamente para analizar opiniones expresadas en texto en redes sociales, noticias y otros tipos de contenido.

"""
import pandas as pd # Cargamos la libreria de "pandas" para la manipulación y el análisis de datos
import nltk #Cargaremos nltk para poder usar expresiones regulares
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer # Cargamos la libreria vadersentiment
import re #Importamos re para control de expresiones regulares
nltk.download('punkt') #Instalamos punkt
nltk.download('stopwords') #Instalamos las stopwords
nltk.download('vader_lexicon') #Instalamos vader
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CRISTHIAN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CRISTHIAN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\CRISTHIAN\AppData\Roaming\nltk_data...


## 2. Analisis de Sentimiento (User_Reviews)

### 2.1. Carga de DataSet

In [3]:
#Establecemos las rutas de los archivos
ruta_user_reviews_parquet = r'Dataset_Clean\australian_user_reviews_clean.parquet'

In [4]:
#Cargamos el archivo australian_user_reviews
df_user_reviews = pd.read_parquet(ruta_user_reviews_parquet)

### 2.2. Analisis Descriptivo del DataSet

#### 2.2.1. Definición de las Funciones Descriptivas del DataSet

In [5]:
#Definimos algunas funciones para que nos facilita la descripcion de las principales caracteristicas del DataFrame
def caracteristicas_df(df):
    """
    Describe de forma general la base de datos .

    Esta función simplemente muestra el tamaño, información general y
    la cantidad de datos nulos.

    Parametros
    ----------
    df (pandas.DataFrame): El DataFrame que se va a analizar.

    Returns:
    ----------
        - 'df.shape': Numero de filas y columnas
        - 'df.info': Muestra información general del DataFrame

    """
    print('*'*10 + '|'*10 + 'FORMA DE BASE DE DATOS' + '|'*10 + '*'*10, end = '\n'*2)
    print(f'Tiene {df.shape[0]} filas y {df.shape[1]} columnas o variables')
    print(end = '\n'*2)

    print('*'*10 + '|'*10 + 'INFORMACION GENERAL DE LA BASE DE DATOS' + '|'*10 + '*'*10, end = '\n'*2)
    print(df.info(), end = '\n'*2)

def valores_nulos_df(df):
    """
    Revisa presencia de valores nulos en un DataFrame.
    Esta función toma un DataFrame como entrada y devuelve un resumen que incluye información sobre
    el porcentaje de valores no nulos y nulos, así como la ncantidad de valores nulos por columna.

    Parametros:
    ----------
    df (pandas.DataFrame): El DataFrame que se va a analizar.

    Returns:
    ----------
        pandas.DataFrame: Un DataFrame que contiene el resumen de cada columna, incluyendo:
        - 'nombre': Nombre de cada columna.
        - 'no_nulos_%': Porcentaje de valores no nulos en cada columna.
        - 'nulos_%': Porcentaje de valores nulos en cada columna.
        - 'nulos': Cantidad de valores nulos en cada columna.

    """
    mi_df = {"nombre": [], "tipo_datos": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_df["nombre"].append(columna)
        mi_df["tipo_datos"].append(df[columna].apply(type).unique())
        mi_df["nulos_%"].append(round(100-porcentaje_no_nulos, 2))
        mi_df["nulos"].append(df[columna].isnull().sum())

    df_nulos = pd.DataFrame(mi_df)

    return df_nulos

#### 2.2.2. Descripcion del DataSet

In [6]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet
caracteristicas_df(df_user_reviews)

**********||||||||||FORMA DE BASE DE DATOS||||||||||**********

Tiene 53902 filas y 10 columnas o variables


**********||||||||||INFORMACION GENERAL DE LA BASE DE DATOS||||||||||**********

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53902 entries, 0 to 53901
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   user_id               53902 non-null  object
 1   user_url              53902 non-null  object
 2   item_id               53902 non-null  int64 
 3   recommend             53902 non-null  int64 
 4   review                53902 non-null  object
 5   Posted Date           53902 non-null  object
 6   Date last edited      53902 non-null  object
 7   funny review votes    53902 non-null  int64 
 8   Helpful review votes  53902 non-null  int64 
 9   total review votes    53902 non-null  int64 
dtypes: int64(5), object(5)
memory usage: 4.1+ MB
None



In [7]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet
valores_nulos_df(df_user_reviews)

,nombre,tipo_datos,nulos_%,nulos
0,user_id,[<class 'str'>],0.0,0
1,user_url,[<class 'str'>],0.0,0
2,item_id,[<class 'int'>],0.0,0
3,recommend,[<class 'int'>],0.0,0
4,review,[<class 'str'>],0.0,0
5,Posted Date,[<class 'str'>],0.0,0
6,Date last edited,[<class 'str'>],0.0,0
7,funny review votes,[<class 'int'>],0.0,0
8,Helpful review votes,[<class 'int'>],0.0,0
9,total review votes,[<class 'int'>],0.0,0


### 2.3. Transformación de Datos y Procesamiento de Texto

#### 2.3.1. Funciones de Transformación

In [8]:
def tokenization(text):
    """
    Tokeniza un texto y elimina las stopwords en inglés (a excepción de 'not').

    Parámetros:
    ---------
    text (str): Texto que se va a tokenizar y limpiar.

    Returns:
    ---------
    str: Cadena de texto resultante después de la tokenización y eliminación de stopwords.
    """
    if isinstance(text, str):
        # Obtener la lista de stopwords en inglés excluyendo 'not'
        stopwords_list = nltk.corpus.stopwords.words('english')
        stopwords_list = [word for word in stopwords_list if 'not' not in word]

        # Tokenizar el texto y eliminar stopwords
        tokens = nltk.tokenize.word_tokenize(text)
        tokens = [word for word in tokens if word.lower() not in stopwords_list]

        return ' '.join(tokens)
    else:
        return text


def delete_repeated_characters(text):
    """
    Elimina caracteres repetidos en las palabras del texto.

    Parámetros:
    ---------
    text (str): Texto que se va a procesar para eliminar caracteres repetidos.

    Returns:
    ---------
    str: Cadena de texto resultante después de eliminar caracteres repetidos.
    """
    # Compilar un patrón regex para encontrar caracteres repetidos más de 2 veces
    pattern = re.compile(r'(.)\1{2,}', re.DOTALL)
    # Sustituir caracteres repetidos con el mismo caracter (excepto si es una vocal)
    text_cleaned = pattern.sub(r'\1\1', text)

    return text_cleaned


# Llamamos a la función SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()


def analyze_sentiment(text):
    """
    Realiza análisis de sentimiento en un texto y asigna una etiqueta según la escala.

    Parámetros:
    ---------
    text (str): Texto que se va a analizar.

    Returns:
    ---------
    int: Etiqueta de sentimiento. 0: Negativo, 1: Neutro, 2: Positivo.
    """
    if pd.isnull(text) or text == '':
        return 1  # Devuelve 1 para textos nulos o vacíos (neutral)

    # Obtener puntuaciones de polaridad usando SentimentIntensityAnalyzer
    polarity_scores = sid.polarity_scores(text)
    compound_score = polarity_scores.get('compound')

    # Asignar etiqueta según la puntuación de compuesto
    if compound_score > 0:
        return 2  # Positivo
    elif compound_score < 0:
        return 0  # Negativo
    else:
        return 1  # Neutro


In [9]:
#Aplicamos las funciones correspondientes para hacer el analisis de sentimiento
"""
Esta línea utiliza la función tokenization para procesar cada elemento en la columna 'review'. Aplica tokenización y elimina stopwords
(palabras comunes que generalmente no aportan información significativa al análisis), excluyendo la palabra 'not'.
 El resultado se guarda nuevamente en la columna 'review'.
"""
df_user_reviews['review'] = df_user_reviews['review'].apply(lambda x: tokenization(x))

"""
Aquí, la función delete_repeated_characters se aplica a cada elemento en la columna 'review'. La función elimina caracteres repetidos en las palabras,
manteniendo solo dos repeticiones seguidas del mismo carácter, excepto en el caso de las vocales. El resultado se guarda nuevamente en la columna 'review'.

"""

df_user_reviews['review'] = df_user_reviews['review'].apply(lambda x: delete_repeated_characters(x))

"""
La función analyze_sentiment se aplica a cada elemento en la columna 'review' para realizar un análisis de sentimiento.
El resultado, que es la etiqueta de sentimiento (0 para negativo, 1 para neutro, 2 para positivo), se almacena en una nueva columna llamada 'sentiment_analysis'.

"""

df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(analyze_sentiment)


In [10]:
#Visualizamos las caracteristicas del nuevo DataFrame con la columna agregada
caracteristicas_df(df_user_reviews)

**********||||||||||FORMA DE BASE DE DATOS||||||||||**********

Tiene 53902 filas y 11 columnas o variables


**********||||||||||INFORMACION GENERAL DE LA BASE DE DATOS||||||||||**********

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53902 entries, 0 to 53901
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   user_id               53902 non-null  object
 1   user_url              53902 non-null  object
 2   item_id               53902 non-null  int64 
 3   recommend             53902 non-null  int64 
 4   review                53902 non-null  object
 5   Posted Date           53902 non-null  object
 6   Date last edited      53902 non-null  object
 7   funny review votes    53902 non-null  int64 
 8   Helpful review votes  53902 non-null  int64 
 9   total review votes    53902 non-null  int64 
 10  sentiment_analysis    53902 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 4.5+ MB


In [11]:
#Verificamos si existen algunos datos nulos
valores_nulos_df(df_user_reviews)

,nombre,tipo_datos,nulos_%,nulos
0,user_id,[<class 'str'>],0.0,0
1,user_url,[<class 'str'>],0.0,0
2,item_id,[<class 'int'>],0.0,0
3,recommend,[<class 'int'>],0.0,0
4,review,[<class 'str'>],0.0,0
5,Posted Date,[<class 'str'>],0.0,0
6,Date last edited,[<class 'str'>],0.0,0
7,funny review votes,[<class 'int'>],0.0,0
8,Helpful review votes,[<class 'int'>],0.0,0
9,total review votes,[<class 'int'>],0.0,0


In [12]:
#Visualizamos nuestro nuevo DataFrame
df_user_reviews.head()

,user_id,user_url,item_id,recommend,review,Posted Date,Date last edited,funny review votes,Helpful review votes,total review votes,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,1,simple yet great replayability opinion zombie ...,2011-11-05,2011-11-05,0,0,0,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,1,unique worth playthrough,2011-07-15,2011-07-15,0,0,0,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,1,great atmosphere gunplay bit chunky times end ...,2011-04-21,2011-04-21,0,0,0,2
3,js41637,http://steamcommunity.com/id/js41637,251610,1,know think see title barbie dreamhouse party n...,2014-06-24,2014-06-24,0,15,20,2
4,js41637,http://steamcommunity.com/id/js41637,227300,1,simple actually not simple truck driving simul...,2013-09-08,2013-09-08,0,0,1,2


### 2.4. Exportación de DataSet Limpia

In [13]:
#Se guarda el dataframe transformado como australian_user_reviews_sentanaly_clean
archivo_limpio = r'Dataset_Clean\australian_user_reviews_sentanaly_clean.csv'
df_user_reviews.to_csv(archivo_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {archivo_limpio}')

Se guardó el archivo Dataset_Clean\australian_user_reviews_sentanaly_clean.csv


In [14]:
#Se guarda el dataframe transformado como australian_user_reviews_sentanaly_clean
archivo_limpio_csv = r'Dataset_Clean\australian_user_reviews_sentanaly_clean.csv'
archivo_limpio_parquet = r'Dataset_Clean\australian_user_reviews_sentanaly_clean.parquet'
# Leemos el archivo CSV en un DataFrame
df = pd.read_csv(archivo_limpio_csv)
df.to_parquet(archivo_limpio_parquet, engine='pyarrow')
print(f'Se guardó el archivo {archivo_limpio_parquet}')

Se guardó el archivo Dataset_Clean\australian_user_reviews_sentanaly_clean.parquet
